In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

#Alternative to openAI models
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [2]:
 import pandas as pd

houses = pd.read_csv('house_data_cleaned_v2.csv')
houses

,house_type,price,bedrooms,bathrooms,neighbourhood_quality,house_condition,crime_rate,garden_size,house_keywords,new_words_in_keywords,co-ordinates,tag,house_tag
0,Semi-Detached,1032367.15,3,3,Pleasant,Alright,High,19.67,"['chateau', 'non-toxic-materials', 'secluded',...",69,50.846545 -0.539285,1,"1 ['chateau', 'non-toxic-materials', 'secluded..."
1,Flat,462379.82,3,1,Rough,Alright,Medium,951.33,"['basement', 'workshop', 'art-studio', 'scenic...",35,50.894080 -2.750017,2,"2 ['basement', 'workshop', 'art-studio', 'scen..."
2,Terraced,225315.47,1,1,Pleasant,Abysmal,Very Low,156.70,"['airy', 'organic-gardening', 'exposed-beams',...",48,53.541494 -2.099826,3,"3 ['airy', 'organic-gardening', 'exposed-beams..."
3,Terraced,763644.90,5,2,Pleasant,Alright,Low,6.49,"['community-garden', 'ecosystem-friendly', 'ap...",94,51.414573 -0.370068,4,"4 ['community-garden', 'ecosystem-friendly', '..."
4,Bungalow,79402.78,6,1,Pleasant,Alright,Medium,253.48,"['dog-run', 'municipal-water', 'radon-mitigati...",28,54.900993 -1.393848,5,"5 ['dog-run', 'municipal-water', 'radon-mitiga..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2616,Terraced,370515.03,3,2,Pleasant,Alright,Medium,52.57,"['courtyard', 'flood-resistant', 'workshop', '...",57,57.184340 -2.181028,2617,"2617 ['courtyard', 'flood-resistant', 'worksho..."
2617,Terraced,685726.47,3,2,Pleasant,Pleasant,Very High,9.18,"['carpeted', 'stone-wall', 'ample-storage', 'f...",64,53.899039 -1.738359,2618,"2618 ['carpeted', 'stone-wall', 'ample-storage..."
2618,Terraced,106448.44,6,4,Outstanding,Alright,Medium,21.51,"['vegetable-garden', 'outdoor-kitchen', 'rural...",53,51.491219 0.163004,2619,"2619 ['vegetable-garden', 'outdoor-kitchen', '..."
2619,Flat,200903.87,5,1,Alright,Alright,Medium,6.33,"['comfortable', 'partly-furnished', 'villa', '...",88,51.322855 -0.547465,2620,"2620 ['comfortable', 'partly-furnished', 'vill..."


In [3]:
houses["house_tag"]

0       1 ['chateau', 'non-toxic-materials', 'secluded...
1       2 ['basement', 'workshop', 'art-studio', 'scen...
2       3 ['airy', 'organic-gardening', 'exposed-beams...
3       4 ['community-garden', 'ecosystem-friendly', '...
4       5 ['dog-run', 'municipal-water', 'radon-mitiga...
                              ...                        
2616    2617 ['courtyard', 'flood-resistant', 'worksho...
2617    2618 ['carpeted', 'stone-wall', 'ample-storage...
2618    2619 ['vegetable-garden', 'outdoor-kitchen', '...
2619    2620 ['comfortable', 'partly-furnished', 'vill...
2620    2621 ['barbecue-area', 'butler-pantry', 'zero-...
Name: house_tag, Length: 2621, dtype: object

In [4]:
houses["house_tag"].to_csv("house_tag.txt",
                              sep = "\n",
                              index = False,
                              header = False)

In [5]:
def extract_keywords(text):
    # Split on the first occurrence of '[' to isolate the keywords part.
    parts = text.split('[', 1)
    if len(parts) > 1:
        # Return the keywords part with the '[' added back.
        return '[' + parts[1]
    return text

In [6]:
#Load
raw_documents = TextLoader("house_tag.txt").load()

#Instantiate splitting
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")

#Apply splitting to loaded document
documents = text_splitter.split_documents(raw_documents)

# Process each document to extract only the keywords portion.
keywords_only = [extract_keywords(doc.page_content) for doc in documents]

Created a chunk of size 1090, which is longer than the specified 0
Created a chunk of size 510, which is longer than the specified 0
Created a chunk of size 708, which is longer than the specified 0
Created a chunk of size 1465, which is longer than the specified 0
Created a chunk of size 456, which is longer than the specified 0
Created a chunk of size 1461, which is longer than the specified 0
Created a chunk of size 744, which is longer than the specified 0
Created a chunk of size 659, which is longer than the specified 0
Created a chunk of size 694, which is longer than the specified 0
Created a chunk of size 709, which is longer than the specified 0
Created a chunk of size 501, which is longer than the specified 0
Created a chunk of size 1148, which is longer than the specified 0
Created a chunk of size 915, which is longer than the specified 0
Created a chunk of size 1591, which is longer than the specified 0
Created a chunk of size 1265, which is longer than the specified 0
Crea

In [7]:
print(keywords_only[0])

['chateau', 'non-toxic-materials', 'secluded', 'suburban', 'greywater-system', 'rain-garden', 'eco-conscious', 'wildlife-habitat', 'fish-pond', 'chandelier', 'private', 'chef-kitchen', 'radon-mitigation', 'media-room', 'carport', 'walk-in-closet', 'backyard', 'mountain-cabin', 'bicycle-storage', 'renovated', 'quadplex', 'garage', 'thermal-insulation', 'sustainable-lifestyle', 'yoga-studio', 'ecosystem-friendly', 'double-glazed', 'reception', 'flower-beds', 'seasonal-living', 'minimalist', 'workshop', 'contemporary', 'quiet', 'solar-panels', 'breathtaking', 'gas-fireplace', 'carbon-monoxide-detectors', 'mudroom', 'sprinkler-system', 'composting-toilet', 'geothermal-heating', 'modern', 'airy', 'foyer', 'charming', 'soundproof', 'wildflower-meadow', 'pantry', 'vintage-style', 'water-softener', 'panoramic-view', 'fireplace', 'traditional', 'zero-waste', 'atrium', 'picture-window', 'energy-efficient', 'drought-tolerant', 'high-ceiling', 'vaulted-ceiling', 'well-maintained', 'green-living', 

In [8]:
documents[0]

Document(metadata={'source': 'house_tag.txt'}, page_content="1 ['chateau', 'non-toxic-materials', 'secluded', 'suburban', 'greywater-system', 'rain-garden', 'eco-conscious', 'wildlife-habitat', 'fish-pond', 'chandelier', 'private', 'chef-kitchen', 'radon-mitigation', 'media-room', 'carport', 'walk-in-closet', 'backyard', 'mountain-cabin', 'bicycle-storage', 'renovated', 'quadplex', 'garage', 'thermal-insulation', 'sustainable-lifestyle', 'yoga-studio', 'ecosystem-friendly', 'double-glazed', 'reception', 'flower-beds', 'seasonal-living', 'minimalist', 'workshop', 'contemporary', 'quiet', 'solar-panels', 'breathtaking', 'gas-fireplace', 'carbon-monoxide-detectors', 'mudroom', 'sprinkler-system', 'composting-toilet', 'geothermal-heating', 'modern', 'airy', 'foyer', 'charming', 'soundproof', 'wildflower-meadow', 'pantry', 'vintage-style', 'water-softener', 'panoramic-view', 'fireplace', 'traditional', 'zero-waste', 'atrium', 'picture-window', 'energy-efficient', 'drought-tolerant', 'high-c

In [11]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
print(model.encode("Test sentence."))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[ 6.61363006e-02  7.42782578e-02 -5.43724233e-03  1.07024081e-01
 -1.50588108e-02 -1.59954913e-02  5.93422353e-02  2.58089397e-02
 -3.10458634e-02  5.06792974e-04  1.35033712e-01 -3.03744134e-02
  9.04069003e-03  2.61266343e-02  3.79480943e-02 -3.00213993e-02
  3.27411145e-02 -6.29581735e-02 -7.55994543e-02  3.39907221e-02
  5.08246087e-02  3.13840583e-02  1.63659025e-02  2.98613720e-02
  2.03087199e-02  5.38612679e-02 -7.36508965e-02  3.25585566e-02
  6.31185696e-02 -1.01804072e-02 -2.47850530e-02  1.91815682e-02
  1.84100531e-02  5.41303605e-02  2.57337447e-02  6.76014228e-03
  8.18789192e-03  1.54630691e-02  5.58992499e-04 -2.96843052e-02
  1.19804721e-02 -1.00487866e-01  7.06751123e-02 -7.45055079e-03
  2.46165171e-02  8.17521568e-03 -2.08241921e-02  4.31874171e-02
 -5.05968146e-02 -3.77045758e-02 -3.70134264e-02 -6.11897744e-02
 -5.80991618e-02  1.04094790e-02 -1.16885118e-02  1.15054594e-02
  2.29975656e-02  9.66378301e-03 -4.94160317e-03  1.61723082e-03
  6.62664138e-03 -3.40837

In [9]:
#Build the vector database
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db_house = Chroma.from_documents(documents, embedding=embeddings)

C:\Users\GODIAH\AppData\Local\Temp\ipykernel_5932\4074314782.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [10]:
query = "I am looking for a house with a jacuzzi, a pantry, sustainable and eco-friendly with solar panels, have high ceilings and an open floor plan. Also a balcony could be nice"
results = db_house.similarity_search(query, k=3)
results

[Document(metadata={'source': 'house_tag.txt'}, page_content="1362 ['fire-escape', 'jacuzzi', 'breakfast-nook', 'hobby-room', 'single-family', 'asbestos-free', 'bicycle-storage', 'partly-furnished', 'attic', 'utility-room', 'sauna', 'gas-stove', 'xeriscaping', 'exposed-beams', 'heated-floors', 'high-ceiling', 'biodynamic', 'condo', 'soundproof', 'rain-garden', 'CCTV', 'eco-friendly', 'storage-space', 'convenient', 'en-suite', 'townhouse', 'homely', 'hurricane-proof', 'chandelier', 'ample-storage', 'gas-fireplace', 'quiet', 'wooden-floors', 'comfortable', 'picturesque', 'multi-family', 'balcony', 'skyline-view', 'master-bedroom', 'sustainable-lifestyle', 'energy-saving']"),
 Document(metadata={'source': 'house_tag.txt'}, page_content="533 ['workshop', 'quiet', 'furnished', 'shutters', 'native-plants', 'beach-house', 'backyard', 'walk-in-closet', 'media-room', 'atrium', 'foyer', 'smoke-detectors', 'food-forest', 'water-efficient', 'solar-shades', 'functional', 'rustic', 'insulated', 'saf

In [11]:
# Retrieve the house details
houses[houses["tag"] == int(results[0].page_content.split()[0].strip())]

,house_type,price,bedrooms,bathrooms,neighbourhood_quality,house_condition,crime_rate,garden_size,house_keywords,new_words_in_keywords,co-ordinates,tag,house_tag
1361,Terraced,667692.44,5,2,Pleasant,Alright,Medium,618.56,"['fire-escape', 'jacuzzi', 'breakfast-nook', '...",41,52.462907 -1.876248,1362,"1362 ['fire-escape', 'jacuzzi', 'breakfast-noo..."


In [12]:
# Retrieve recommendation function
def retrieve_house_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_house.similarity_search(query, k = 50)

    houses_list = []

    for i in range(0, len(recs)):
        houses_list += [int(recs[i].page_content.strip('"').split()[0])]
    return houses[houses["tag"].isin(houses_list)].head(top_k)

In [13]:
retrieve_house_recommendations("A house with open floor planning, high ceilings, built in wardrobes and air conditioning")

,house_type,price,bedrooms,bathrooms,neighbourhood_quality,house_condition,crime_rate,garden_size,house_keywords,new_words_in_keywords,co-ordinates,tag,house_tag
1,Flat,462379.82,3,1,Rough,Alright,Medium,951.33,"['basement', 'workshop', 'art-studio', 'scenic...",35,50.894080 -2.750017,2,"2 ['basement', 'workshop', 'art-studio', 'scen..."
16,Flat,95095.49,5,3,Outstanding,Abysmal,Low,33.86,"['atrium', 'carport', 'peaceful', 'biodynamic'...",68,52.734131 1.465429,17,"17 ['atrium', 'carport', 'peaceful', 'biodynam..."
27,Terraced,245665.44,4,3,Abysmal,Abysmal,Low,7.55,"['art-deco', 'island-kitchen', 'luxurious', 'p...",65,54.887123 -1.370081,28,"28 ['art-deco', 'island-kitchen', 'luxurious',..."
117,Flat,509075.08,5,2,Pleasant,Alright,Low,85.49,"['air-conditioning', 'HEPA-filters', 'sliding-...",34,51.381273 -0.840669,118,"118 ['air-conditioning', 'HEPA-filters', 'slid..."
118,Flat,65865.66,4,1,Alright,Alright,Medium,6.89,"['energy-efficient', 'laundry-room', 'farmhous...",89,50.379174 -4.137756,119,"119 ['energy-efficient', 'laundry-room', 'farm..."
178,Detached,154763.20,1,2,Rough,Alright,Medium,95.92,"['furnished', 'flat', 'exposed-beams', 'home-o...",97,55.990463 -3.970225,179,"179 ['furnished', 'flat', 'exposed-beams', 'ho..."
419,Semi-Detached,224102.84,2,1,Outstanding,Alright,Low,54.75,"['charming', 'en-suite', 'energy-star', 'air-c...",84,51.605816 0.160895,420,"420 ['charming', 'en-suite', 'energy-star', 'a..."
479,Detached,1065468.03,5,1,Alright,Abysmal,Very High,497.81,"['vintage', 'landscaped', 'organic-gardening',...",33,51.540172 -0.196193,480,"480 ['vintage', 'landscaped', 'organic-gardeni..."
493,Terraced,156945.99,4,1,Alright,Rough,Very Low,49.08,"['farmhouse-sink', 'art-studio', 'loft', 'heat...",26,51.499438 -3.185582,494,"494 ['farmhouse-sink', 'art-studio', 'loft', '..."
494,Bungalow,363984.09,1,3,Pleasant,Alright,Medium,4.22,"['barbecue-area', 'tiled-floors', 'passive-hou...",40,51.314921 0.892582,495,"495 ['barbecue-area', 'tiled-floors', 'passive..."


In [14]:
retrieve_house_recommendations("I want a house with a pantry, barbecue spot, pet-friendly and is furnished")

,house_type,price,bedrooms,bathrooms,neighbourhood_quality,house_condition,crime_rate,garden_size,house_keywords,new_words_in_keywords,co-ordinates,tag,house_tag
53,Semi-Detached,306146.52,2,2,Rough,Alright,Low,61.66,"['gas-stove', 'safe', 'smart-home', 'compostin...",84,53.005204 -1.117606,54,"54 ['gas-stove', 'safe', 'smart-home', 'compos..."
59,Flat,662049.81,3,1,Pleasant,Alright,Low,1404.11,"['partly-furnished', 'quiet', 'penthouse', 'gr...",60,52.671355 -2.420272,60,"60 ['partly-furnished', 'quiet', 'penthouse', ..."
93,Terraced,243678.85,3,3,Outstanding,Alright,Medium,52.29,"['storm-shutters', 'indoor-air-quality', 'dupl...",58,51.563769 0.043314,94,"94 ['storm-shutters', 'indoor-air-quality', 'd..."
101,Semi-Detached,668811.36,3,3,Pleasant,Pleasant,Medium,84.03,"['zero-waste', 'convenient', 'comfortable', 'c...",39,54.948078 -1.970900,102,"102 ['zero-waste', 'convenient', 'comfortable'..."
184,Flat,62373.83,2,2,Alright,Abysmal,Low,120.00,"['playground', 'dining-room', 'pest-control', ...",44,50.758188 -1.924383,185,"185 ['playground', 'dining-room', 'pest-contro..."
231,Semi-Detached,140448.47,5,1,Outstanding,Alright,Non Existent,3.14,"['patio', 'family-friendly', 'contemporary-des...",41,53.707110 -1.151864,232,"232 ['patio', 'family-friendly', 'contemporary..."
251,Flat,217314.83,4,1,Pleasant,Alright,Low,199.11,"['peaceful', 'vaulted-ceiling', 'lead-free', '...",85,53.592022 -2.482116,252,"252 ['peaceful', 'vaulted-ceiling', 'lead-free..."
264,Bungalow,232162.33,4,3,Alright,Pleasant,Low,87.63,"['pet-friendly', 'contemporary-design', 'quadp...",33,51.827374 -0.222368,265,"265 ['pet-friendly', 'contemporary-design', 'q..."
378,Detached,377000.56,2,4,Rough,Rough,Very Low,116.76,"['breathtaking', 'furnished', 'barbecue-area',...",86,52.391669 -1.977326,379,"379 ['breathtaking', 'furnished', 'barbecue-ar..."
422,Terraced,506813.27,3,4,Alright,Alright,Extreme,529.99,"['chandelier', 'pest-control', 'scenic', 'cent...",84,51.378068 -0.113266,423,"423 ['chandelier', 'pest-control', 'scenic', '..."
